In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [3]:
# get file paths in a list
file_paths = [r'C:\Users\arulkumar\Desktop\MLGuvi\Singapore_Resale_Flat_Prices_Predicting\ResaleFlatPricesBasedonApprovalDate2000Feb2012.csv',r'C:\Users\arulkumar\Desktop\MLGuvi\Singapore_Resale_Flat_Prices_Predicting\ResaleFlatPricesBasedonApprovalDate19901999.csv', r'C:\Users\arulkumar\Desktop\MLGuvi\Singapore_Resale_Flat_Prices_Predicting\ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv', r'C:\Users\arulkumar\Desktop\MLGuvi\Singapore_Resale_Flat_Prices_Predicting\ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv', r'C:\Users\arulkumar\Desktop\MLGuvi\Singapore_Resale_Flat_Prices_Predicting\ResaleFlatPricesBasedonRegistrationDateFromMar2012toDec2014.csv']
# declare an empty list
dfs = []
# append each csv to the dfs list
for file_path in file_paths:
    df = pd.read_csv(file_path)
    dfs.append(df)
# combine them using concat
combined_df = pd.concat(dfs, ignore_index=True)
     

In [6]:
# display the combined dataframe
combined_df.tail()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
911590,2014-12,YISHUN,5 ROOM,816,YISHUN ST 81,10 TO 12,122.0,Improved,1988,580000.0,NaN
911591,2014-12,YISHUN,EXECUTIVE,325,YISHUN CTRL,10 TO 12,146.0,Maisonette,1988,540000.0,NaN
911592,2014-12,YISHUN,EXECUTIVE,618,YISHUN RING RD,07 TO 09,164.0,Apartment,1992,738000.0,NaN
911593,2014-12,YISHUN,EXECUTIVE,277,YISHUN ST 22,07 TO 09,152.0,Maisonette,1985,592000.0,NaN
911594,2014-12,YISHUN,EXECUTIVE,277,YISHUN ST 22,04 TO 06,146.0,Maisonette,1985,545000.0,NaN


In [7]:
# check for null values
combined_df.isnull().sum()

month                       0
town                        0
flat_type                   0
block                       0
street_name                 0
storey_range                0
floor_area_sqm              0
flat_model                  0
lease_commence_date         0
resale_price                0
remaining_lease        709050
dtype: int64

In [8]:
# check the number of streets
street_nos = len(combined_df['street_name'].unique())
street_nos

582

In [9]:
# list unique street names
streets = combined_df['street_name'].unique()
streets

array(['ANG MO KIO AVE 4', 'ANG MO KIO AVE 1', 'ANG MO KIO AVE 10',
       'ANG MO KIO AVE 2', 'ANG MO KIO AVE 3', 'ANG MO KIO AVE 5',
       'ANG MO KIO AVE 6', 'ANG MO KIO AVE 8', 'ANG MO KIO AVE 9',
       'ANG MO KIO ST 21', 'ANG MO KIO ST 31', 'ANG MO KIO ST 11',
       'BEDOK RESERVOIR RD', 'BEDOK NTH RD', 'BEDOK NTH ST 3',
       'BEDOK STH AVE 1', 'BEDOK STH RD', 'CHAI CHEE AVE',
       'NEW UPP CHANGI RD', 'CHAI CHEE DR', 'CHAI CHEE RD',
       'BEDOK NTH AVE 1', 'BEDOK NTH AVE 2', 'BEDOK NTH AVE 3',
       'BEDOK NTH AVE 4', 'BEDOK NTH ST 1', 'BEDOK NTH ST 2',
       'BEDOK NTH ST 4', 'BEDOK STH AVE 2', 'BEDOK STH AVE 3',
       'CHAI CHEE ST', 'LENGKONG TIGA', 'JLN TENAGA', 'SIN MING RD',
       'SHUNFU RD', 'BISHAN ST 11', 'BISHAN ST 12', 'BISHAN ST 13',
       'SIN MING AVE', 'BISHAN ST 22', 'BISHAN ST 23', 'BISHAN ST 24',
       'BRIGHT HILL DR', 'BT BATOK ST 51', 'BT BATOK ST 52',
       'BT BATOK ST 11', 'BT BATOK ST 21', 'BT BATOK ST 34',
       'BT BATOK WEST AVE 4', 

In [10]:
# map the streets to povide input to ML model
streets_mapping = {street: idx + 1 for idx, street in enumerate(streets)}
streets_mapping

{'ANG MO KIO AVE 4': 1,
 'ANG MO KIO AVE 1': 2,
 'ANG MO KIO AVE 10': 3,
 'ANG MO KIO AVE 2': 4,
 'ANG MO KIO AVE 3': 5,
 'ANG MO KIO AVE 5': 6,
 'ANG MO KIO AVE 6': 7,
 'ANG MO KIO AVE 8': 8,
 'ANG MO KIO AVE 9': 9,
 'ANG MO KIO ST 21': 10,
 'ANG MO KIO ST 31': 11,
 'ANG MO KIO ST 11': 12,
 'BEDOK RESERVOIR RD': 13,
 'BEDOK NTH RD': 14,
 'BEDOK NTH ST 3': 15,
 'BEDOK STH AVE 1': 16,
 'BEDOK STH RD': 17,
 'CHAI CHEE AVE': 18,
 'NEW UPP CHANGI RD': 19,
 'CHAI CHEE DR': 20,
 'CHAI CHEE RD': 21,
 'BEDOK NTH AVE 1': 22,
 'BEDOK NTH AVE 2': 23,
 'BEDOK NTH AVE 3': 24,
 'BEDOK NTH AVE 4': 25,
 'BEDOK NTH ST 1': 26,
 'BEDOK NTH ST 2': 27,
 'BEDOK NTH ST 4': 28,
 'BEDOK STH AVE 2': 29,
 'BEDOK STH AVE 3': 30,
 'CHAI CHEE ST': 31,
 'LENGKONG TIGA': 32,
 'JLN TENAGA': 33,
 'SIN MING RD': 34,
 'SHUNFU RD': 35,
 'BISHAN ST 11': 36,
 'BISHAN ST 12': 37,
 'BISHAN ST 13': 38,
 'SIN MING AVE': 39,
 'BISHAN ST 22': 40,
 'BISHAN ST 23': 41,
 'BISHAN ST 24': 42,
 'BRIGHT HILL DR': 43,
 'BT BATOK ST 51': 

In [11]:
# alter the column with the mapped numbers
combined_df['street_name'] = combined_df['street_name'].map(streets_mapping)

# Display the DataFrame
combined_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,2000-01,ANG MO KIO,3 ROOM,170,1,07 TO 09,69.0,Improved,1986,147000.0,NaN
1,2000-01,ANG MO KIO,3 ROOM,174,1,04 TO 06,61.0,Improved,1986,144000.0,NaN
2,2000-01,ANG MO KIO,3 ROOM,216,2,07 TO 09,73.0,New Generation,1976,159000.0,NaN
3,2000-01,ANG MO KIO,3 ROOM,215,2,07 TO 09,73.0,New Generation,1976,167000.0,NaN
4,2000-01,ANG MO KIO,3 ROOM,218,2,07 TO 09,67.0,New Generation,1976,163000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
911590,2014-12,YISHUN,5 ROOM,816,327,10 TO 12,122.0,Improved,1988,580000.0,NaN
911591,2014-12,YISHUN,EXECUTIVE,325,322,10 TO 12,146.0,Maisonette,1988,540000.0,NaN
911592,2014-12,YISHUN,EXECUTIVE,618,323,07 TO 09,164.0,Apartment,1992,738000.0,NaN
911593,2014-12,YISHUN,EXECUTIVE,277,325,07 TO 09,152.0,Maisonette,1985,592000.0,NaN


In [12]:
# function to convert the years and months to years in decimal
def convert_to_years(row):
    if pd.notna(row):
        if isinstance(row, str):
            if 'years' in row:
                parts = row.split()
                years = float(parts[0])
                if len(parts) > 2:  # If there's a month part as well
                    months = float(parts[2])
                    return years + months / 12
                else:
                    return years
        else:
          return row
    return row

In [13]:
# Apply the conversion function to the DataFrame column
combined_df['remaining_lease'] = combined_df['remaining_lease'].apply(convert_to_years)
combined_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,2000-01,ANG MO KIO,3 ROOM,170,1,07 TO 09,69.0,Improved,1986,147000.0,NaN
1,2000-01,ANG MO KIO,3 ROOM,174,1,04 TO 06,61.0,Improved,1986,144000.0,NaN
2,2000-01,ANG MO KIO,3 ROOM,216,2,07 TO 09,73.0,New Generation,1976,159000.0,NaN
3,2000-01,ANG MO KIO,3 ROOM,215,2,07 TO 09,73.0,New Generation,1976,167000.0,NaN
4,2000-01,ANG MO KIO,3 ROOM,218,2,07 TO 09,67.0,New Generation,1976,163000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
911590,2014-12,YISHUN,5 ROOM,816,327,10 TO 12,122.0,Improved,1988,580000.0,NaN
911591,2014-12,YISHUN,EXECUTIVE,325,322,10 TO 12,146.0,Maisonette,1988,540000.0,NaN
911592,2014-12,YISHUN,EXECUTIVE,618,323,07 TO 09,164.0,Apartment,1992,738000.0,NaN
911593,2014-12,YISHUN,EXECUTIVE,277,325,07 TO 09,152.0,Maisonette,1985,592000.0,NaN


In [14]:
# fill null values in remaining_lease with its mode
combined_df['remaining_lease']= combined_df['remaining_lease'].fillna(combined_df['remaining_lease'].mode()[0])
combined_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,2000-01,ANG MO KIO,3 ROOM,170,1,07 TO 09,69.0,Improved,1986,147000.0,68.0
1,2000-01,ANG MO KIO,3 ROOM,174,1,04 TO 06,61.0,Improved,1986,144000.0,68.0
2,2000-01,ANG MO KIO,3 ROOM,216,2,07 TO 09,73.0,New Generation,1976,159000.0,68.0
3,2000-01,ANG MO KIO,3 ROOM,215,2,07 TO 09,73.0,New Generation,1976,167000.0,68.0
4,2000-01,ANG MO KIO,3 ROOM,218,2,07 TO 09,67.0,New Generation,1976,163000.0,68.0
...,...,...,...,...,...,...,...,...,...,...,...
911590,2014-12,YISHUN,5 ROOM,816,327,10 TO 12,122.0,Improved,1988,580000.0,68.0
911591,2014-12,YISHUN,EXECUTIVE,325,322,10 TO 12,146.0,Maisonette,1988,540000.0,68.0
911592,2014-12,YISHUN,EXECUTIVE,618,323,07 TO 09,164.0,Apartment,1992,738000.0,68.0
911593,2014-12,YISHUN,EXECUTIVE,277,325,07 TO 09,152.0,Maisonette,1985,592000.0,68.0


In [15]:
# Split the 'Range' column into lower and upper bounds
combined_df[['storey_lower_bound', 'storey_upper_bound']] = combined_df['storey_range'].str.split(' TO ', expand=True)

# Convert the lower and upper bounds to numeric values
combined_df['storey_lower_bound'] = pd.to_numeric(combined_df['storey_lower_bound'])
combined_df['storey_upper_bound'] = pd.to_numeric(combined_df['storey_upper_bound'])

combined_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_lower_bound,storey_upper_bound
0,2000-01,ANG MO KIO,3 ROOM,170,1,07 TO 09,69.0,Improved,1986,147000.0,68.0,7,9
1,2000-01,ANG MO KIO,3 ROOM,174,1,04 TO 06,61.0,Improved,1986,144000.0,68.0,4,6
2,2000-01,ANG MO KIO,3 ROOM,216,2,07 TO 09,73.0,New Generation,1976,159000.0,68.0,7,9
3,2000-01,ANG MO KIO,3 ROOM,215,2,07 TO 09,73.0,New Generation,1976,167000.0,68.0,7,9
4,2000-01,ANG MO KIO,3 ROOM,218,2,07 TO 09,67.0,New Generation,1976,163000.0,68.0,7,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
911590,2014-12,YISHUN,5 ROOM,816,327,10 TO 12,122.0,Improved,1988,580000.0,68.0,10,12
911591,2014-12,YISHUN,EXECUTIVE,325,322,10 TO 12,146.0,Maisonette,1988,540000.0,68.0,10,12
911592,2014-12,YISHUN,EXECUTIVE,618,323,07 TO 09,164.0,Apartment,1992,738000.0,68.0,7,9
911593,2014-12,YISHUN,EXECUTIVE,277,325,07 TO 09,152.0,Maisonette,1985,592000.0,68.0,7,9


In [16]:
# drop storey range column
combined_df.drop('storey_range', axis=1, inplace=True)
combined_df

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_lower_bound,storey_upper_bound
0,2000-01,ANG MO KIO,3 ROOM,170,1,69.0,Improved,1986,147000.0,68.0,7,9
1,2000-01,ANG MO KIO,3 ROOM,174,1,61.0,Improved,1986,144000.0,68.0,4,6
2,2000-01,ANG MO KIO,3 ROOM,216,2,73.0,New Generation,1976,159000.0,68.0,7,9
3,2000-01,ANG MO KIO,3 ROOM,215,2,73.0,New Generation,1976,167000.0,68.0,7,9
4,2000-01,ANG MO KIO,3 ROOM,218,2,67.0,New Generation,1976,163000.0,68.0,7,9
...,...,...,...,...,...,...,...,...,...,...,...,...
911590,2014-12,YISHUN,5 ROOM,816,327,122.0,Improved,1988,580000.0,68.0,10,12
911591,2014-12,YISHUN,EXECUTIVE,325,322,146.0,Maisonette,1988,540000.0,68.0,10,12
911592,2014-12,YISHUN,EXECUTIVE,618,323,164.0,Apartment,1992,738000.0,68.0,7,9
911593,2014-12,YISHUN,EXECUTIVE,277,325,152.0,Maisonette,1985,592000.0,68.0,7,9


In [17]:
# list unique flat type
combined_df['flat_type'].unique()

array(['3 ROOM', '4 ROOM', '5 ROOM', 'EXECUTIVE', '2 ROOM',
       'MULTI-GENERATION', '1 ROOM', 'MULTI GENERATION'], dtype=object)

In [18]:
# Replace 'MULTI-GENERATION' with 'MULTI GENERATION' in the specified column to avoid duplicate
combined_df['flat_type'] = combined_df['flat_type'].str.replace('MULTI-GENERATION', 'MULTI GENERATION')

In [19]:
# list unique flat type to recheck 
combined_df['flat_type'].unique()

array(['3 ROOM', '4 ROOM', '5 ROOM', 'EXECUTIVE', '2 ROOM',
       'MULTI GENERATION', '1 ROOM'], dtype=object)

In [20]:
# Define a mapping of flat_type to numbers
category_mapping = {
    '1 ROOM': 1,
    '2 ROOM': 2,
    '3 ROOM': 3,
    '4 ROOM': 4,
    '5 ROOM': 5,
    'EXECUTIVE': 6,
    'MULTI GENERATION': 7
}

# alter the column with the mapped numbers
combined_df['flat_type'] = combined_df['flat_type'].map(category_mapping)

# Display the DataFrame
combined_df

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_lower_bound,storey_upper_bound
0,2000-01,ANG MO KIO,3,170,1,69.0,Improved,1986,147000.0,68.0,7,9
1,2000-01,ANG MO KIO,3,174,1,61.0,Improved,1986,144000.0,68.0,4,6
2,2000-01,ANG MO KIO,3,216,2,73.0,New Generation,1976,159000.0,68.0,7,9
3,2000-01,ANG MO KIO,3,215,2,73.0,New Generation,1976,167000.0,68.0,7,9
4,2000-01,ANG MO KIO,3,218,2,67.0,New Generation,1976,163000.0,68.0,7,9
...,...,...,...,...,...,...,...,...,...,...,...,...
911590,2014-12,YISHUN,5,816,327,122.0,Improved,1988,580000.0,68.0,10,12
911591,2014-12,YISHUN,6,325,322,146.0,Maisonette,1988,540000.0,68.0,10,12
911592,2014-12,YISHUN,6,618,323,164.0,Apartment,1992,738000.0,68.0,7,9
911593,2014-12,YISHUN,6,277,325,152.0,Maisonette,1985,592000.0,68.0,7,9


In [21]:
# list unique flat model
combined_df['flat_model'].unique()

array(['Improved', 'New Generation', 'Model A', 'Standard', 'Apartment',
       'Simplified', 'Model A-Maisonette', 'Maisonette',
       'Multi Generation', 'Adjoined flat', 'Premium Apartment',
       'Terrace', 'Improved-Maisonette', 'Premium Maisonette', '2-room',
       'Model A2', 'IMPROVED', 'NEW GENERATION', 'MODEL A', 'STANDARD',
       'SIMPLIFIED', 'MODEL A-MAISONETTE', 'APARTMENT', 'MAISONETTE',
       'TERRACE', '2-ROOM', 'IMPROVED-MAISONETTE', 'MULTI GENERATION',
       'PREMIUM APARTMENT', 'Type S1', 'Type S2', 'DBSS',
       'Premium Apartment Loft', '3Gen'], dtype=object)

In [22]:
# Create a mapping of 'flat_model' values to numbers
flat_model_mapping = {
    'IMPROVED': 1,
    'NEW GENERATION': 2,
    'MODEL A': 3,
    'STANDARD': 4,
    'SIMPLIFIED': 5,
    'MODEL A-MAISONETTE': 6,
    'APARTMENT': 7,
    'MAISONETTE': 8,
    'TERRACE': 9,
    '2-ROOM': 10,
    'IMPROVED-MAISONETTE': 11,
    'MULTI GENERATION': 12,
    'PREMIUM APARTMENT': 13,
    'Improved': 14,
    'New Generation': 15,
    'Model A': 16,
    'Standard': 17,
    'Apartment': 18,
    'Simplified': 19,
    'Model A-Maisonette': 20,
    'Maisonette': 21,
    'Multi Generation': 22,
    'Adjoined flat': 23,
    'Premium Apartment': 24,
    'Terrace': 25,
    'Improved-Maisonette': 26,
    'Premium Maisonette': 27,
    '2-room': 28,
    'Model A2': 29,
    'DBSS': 30,
    'Type S1': 31,
    'Type S2': 32,
    'Premium Apartment Loft': 33,
    '3Gen': 34
}

# Map 'flat_model' values to numbers
combined_df['flat_model'] = combined_df['flat_model'].map(flat_model_mapping)

# Display the DataFrame
combined_df

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_lower_bound,storey_upper_bound
0,2000-01,ANG MO KIO,3,170,1,69.0,14,1986,147000.0,68.0,7,9
1,2000-01,ANG MO KIO,3,174,1,61.0,14,1986,144000.0,68.0,4,6
2,2000-01,ANG MO KIO,3,216,2,73.0,15,1976,159000.0,68.0,7,9
3,2000-01,ANG MO KIO,3,215,2,73.0,15,1976,167000.0,68.0,7,9
4,2000-01,ANG MO KIO,3,218,2,67.0,15,1976,163000.0,68.0,7,9
...,...,...,...,...,...,...,...,...,...,...,...,...
911590,2014-12,YISHUN,5,816,327,122.0,14,1988,580000.0,68.0,10,12
911591,2014-12,YISHUN,6,325,322,146.0,21,1988,540000.0,68.0,10,12
911592,2014-12,YISHUN,6,618,323,164.0,18,1992,738000.0,68.0,7,9
911593,2014-12,YISHUN,6,277,325,152.0,21,1985,592000.0,68.0,7,9


In [23]:
# list unique town names
combined_df['town'].unique()

array(['ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH',
       'BUKIT PANJANG', 'BUKIT TIMAH', 'CENTRAL AREA', 'CHOA CHU KANG',
       'CLEMENTI', 'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST',
       'KALLANG/WHAMPOA', 'MARINE PARADE', 'PASIR RIS', 'QUEENSTOWN',
       'SENGKANG', 'SERANGOON', 'TAMPINES', 'TOA PAYOH', 'WOODLANDS',
       'YISHUN', 'SEMBAWANG', 'PUNGGOL', 'LIM CHU KANG'], dtype=object)

In [24]:
# Create a mapping of 'town' values to numbers
town_mapping = {
    'ANG MO KIO': 1,
    'BEDOK': 2,
    'BISHAN': 3,
    'BUKIT BATOK': 4,
    'BUKIT MERAH': 5,
    'BUKIT TIMAH': 6,
    'CENTRAL AREA': 7,
    'CHOA CHU KANG': 8,
    'CLEMENTI': 9,
    'GEYLANG': 10,
    'HOUGANG': 11,
    'JURONG EAST': 12,
    'JURONG WEST': 13,
    'KALLANG/WHAMPOA': 14,
    'MARINE PARADE': 15,
    'QUEENSTOWN': 16,
    'SENGKANG': 17,
    'SERANGOON': 18,
    'TAMPINES': 19,
    'TOA PAYOH': 20,
    'WOODLANDS': 21,
    'YISHUN': 22,
    'LIM CHU KANG': 23,
    'SEMBAWANG': 24,
    'BUKIT PANJANG': 25,
    'PASIR RIS': 26,
    'PUNGGOL': 27
}

# Map 'town' values to numbers
combined_df['town'] = combined_df['town'].map(town_mapping)

# Display the DataFrame
combined_df

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_lower_bound,storey_upper_bound
0,2000-01,1,3,170,1,69.0,14,1986,147000.0,68.0,7,9
1,2000-01,1,3,174,1,61.0,14,1986,144000.0,68.0,4,6
2,2000-01,1,3,216,2,73.0,15,1976,159000.0,68.0,7,9
3,2000-01,1,3,215,2,73.0,15,1976,167000.0,68.0,7,9
4,2000-01,1,3,218,2,67.0,15,1976,163000.0,68.0,7,9
...,...,...,...,...,...,...,...,...,...,...,...,...
911590,2014-12,22,5,816,327,122.0,14,1988,580000.0,68.0,10,12
911591,2014-12,22,6,325,322,146.0,21,1988,540000.0,68.0,10,12
911592,2014-12,22,6,618,323,164.0,18,1992,738000.0,68.0,7,9
911593,2014-12,22,6,277,325,152.0,21,1985,592000.0,68.0,7,9


In [25]:
# Define a mapping for letters to decimal values
letter_mapping = {chr(ord('A') + i): f'.{i+1}' for i in range(26)}

# Replace letters with their corresponding decimal values
for letter, decimal in letter_mapping.items():
    combined_df['block'] = combined_df['block'].str.replace(letter, decimal)

# Use pd.to_numeric to convert the column to float and set invalid parsing to NaN
combined_df['block'] = pd.to_numeric(combined_df['block'])

# Display the DataFrame with the column as float values
combined_df

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_lower_bound,storey_upper_bound
0,2000-01,1,3,170.0,1,69.0,14,1986,147000.0,68.0,7,9
1,2000-01,1,3,174.0,1,61.0,14,1986,144000.0,68.0,4,6
2,2000-01,1,3,216.0,2,73.0,15,1976,159000.0,68.0,7,9
3,2000-01,1,3,215.0,2,73.0,15,1976,167000.0,68.0,7,9
4,2000-01,1,3,218.0,2,67.0,15,1976,163000.0,68.0,7,9
...,...,...,...,...,...,...,...,...,...,...,...,...
911590,2014-12,22,5,816.0,327,122.0,14,1988,580000.0,68.0,10,12
911591,2014-12,22,6,325.0,322,146.0,21,1988,540000.0,68.0,10,12
911592,2014-12,22,6,618.0,323,164.0,18,1992,738000.0,68.0,7,9
911593,2014-12,22,6,277.0,325,152.0,21,1985,592000.0,68.0,7,9


In [26]:
# check datatype of the lease resale month column
month = combined_df['month'].dtype
month

dtype('O')

In [27]:
# Separate the year and month of resale
combined_df[['resale_year', 'resale_month']] = combined_df['month'].str.split('-', expand=True)
combined_df

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_lower_bound,storey_upper_bound,resale_year,resale_month
0,2000-01,1,3,170.0,1,69.0,14,1986,147000.0,68.0,7,9,2000,01
1,2000-01,1,3,174.0,1,61.0,14,1986,144000.0,68.0,4,6,2000,01
2,2000-01,1,3,216.0,2,73.0,15,1976,159000.0,68.0,7,9,2000,01
3,2000-01,1,3,215.0,2,73.0,15,1976,167000.0,68.0,7,9,2000,01
4,2000-01,1,3,218.0,2,67.0,15,1976,163000.0,68.0,7,9,2000,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911590,2014-12,22,5,816.0,327,122.0,14,1988,580000.0,68.0,10,12,2014,12
911591,2014-12,22,6,325.0,322,146.0,21,1988,540000.0,68.0,10,12,2014,12
911592,2014-12,22,6,618.0,323,164.0,18,1992,738000.0,68.0,7,9,2014,12
911593,2014-12,22,6,277.0,325,152.0,21,1985,592000.0,68.0,7,9,2014,12


In [28]:
# drop the resale month column
combined_df.drop('month', axis = 1, inplace= True)
combined_df

,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_lower_bound,storey_upper_bound,resale_year,resale_month
0,1,3,170.0,1,69.0,14,1986,147000.0,68.0,7,9,2000,01
1,1,3,174.0,1,61.0,14,1986,144000.0,68.0,4,6,2000,01
2,1,3,216.0,2,73.0,15,1976,159000.0,68.0,7,9,2000,01
3,1,3,215.0,2,73.0,15,1976,167000.0,68.0,7,9,2000,01
4,1,3,218.0,2,67.0,15,1976,163000.0,68.0,7,9,2000,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
911590,22,5,816.0,327,122.0,14,1988,580000.0,68.0,10,12,2014,12
911591,22,6,325.0,322,146.0,21,1988,540000.0,68.0,10,12,2014,12
911592,22,6,618.0,323,164.0,18,1992,738000.0,68.0,7,9,2014,12
911593,22,6,277.0,325,152.0,21,1985,592000.0,68.0,7,9,2014,12


In [29]:
# check for null values
combined_df.isnull().sum()

town                   0
flat_type              0
block                  0
street_name            0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
remaining_lease        0
storey_lower_bound     0
storey_upper_bound     0
resale_year            0
resale_month           0
dtype: int64

In [30]:
# fetch sample data
c_df = combined_df.sample(n=100000, random_state=42)

In [31]:
# assign feature and target variable
X = c_df.drop('resale_price', axis= 1)
y = c_df['resale_price']

In [32]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train and y_train will be your training data, and X_test and y_test will be your testing data.

In [33]:
# Create and train the Random Forest Regressor model
rf_regressor = RandomForestRegressor(random_state=42)
rf_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_regressor.predict(X_test)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Calculate R-squared (R2) Score
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE)    :", mae)
print("Mean Squared Error (MSE)     :", mse)
print("Root Mean Square Error (RMSE):", rmse)
print("R-squared (R2) Score         :", r2)

Mean Absolute Error (MAE)    : 18819.343450716904
Mean Squared Error (MSE)     : 846564274.2807112
Root Mean Square Error (RMSE): 29095.77760226922
R-squared (R2) Score         : 0.969418258881421


In [34]:
# get sample to predict the resale price
pre_df = combined_df.sample(n=2)
pre_df

,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_lower_bound,storey_upper_bound,resale_year,resale_month
617670,11,4,317.0,153,93.0,2,1983,216000.0,68.000000,4,6,1999,04
744178,19,5,168.1,540,107.0,30,2011,820000.0,90.833333,10,12,2019,05


In [35]:
# drop the resale price column
A = pre_df.drop('resale_price', axis = 1)

In [36]:
# Make predictions on the sample
pre_pred = rf_regressor.predict(A)
pre_pred

array([212268.  , 753573.88])

In [37]:
# save the processed data for the deployment
combined_df.to_csv('c.csv')